<a href="https://colab.research.google.com/github/PAURAB-BHATTACHARJEE/FAKE_NEWS_HINDI/blob/main/Bangla_Fake_News.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import re
import string
# !pip install inltk
# from collections.abc import Iterable
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding,Bidirectional,Conv1D, MaxPooling1D, GlobalMaxPooling1D
from sklearn.model_selection import train_test_split

In [ ]:
# !pip install --upgrade fastai
# !pip install --upgrade inltk

# !pip install abc
# from inltk.inltk import setup

In [ ]:
# from inltk.inltk import setup

In [3]:
fake_csv=pd.read_csv("drive/MyDrive/FAKE_NEWS_DETECTION/bangla_fake.csv")
true_csv=pd.read_csv("drive/MyDrive/FAKE_NEWS_DETECTION/bangla_true.csv")

In [ ]:
fake_csv.head()

,text,label
0,হিজাব বিতর্ক: কর্নাটকের পুরনো ভিডিওকে পশ্চিমবঙ...,0
1,নিউ ইয়র্ক টাইমসের সম্পাদকের মোদী প্রসঙ্গে ভুয়ো...,0
2,কর্নাটক হিজাব বিতর্ক: বাংলাদেশের পুরনো ভিডিও ব...,0
3,কর্নাটক হিজাব বিতর্ক: ভুয়ো দাবিতে মডেলের ফোটোশ...,0
4,হিজাব বিতর্ক: জনতা দল সেকুলার সদস্যার ছবি ভুয়ো...,0


In [ ]:
true_csv.head()

,text,label
0,ক্রিকেট বিশ্বের নতুন চমকের নাম আফগানিস্তান। কয়...,1
1,এশিয়া কাপের ষষ্ঠ ম্যাচে বাংলাদেশ দলের বিপক্ষে ...,1
2,রাজধানীতে মাদক বিরোধী বিশেষ অভিযান পরিচালনা কর...,1
3,সনাতন ধর্মাবলম্বীদের সবচেয়ে বড় ধর্মীয় উৎসব দুর...,1
4,ভারত থেকে পাইপ লাইনের মাধ্যমে সরাসরি দেশে জ্বা...,1


In [ ]:
true_csv.shape

(7107, 2)

In [ ]:
fake_csv.shape

(2667, 2)

In [4]:
# df=pd.concat([true_csv,fake_csv],axis=0)

In [5]:
# Determine the size of the smaller dataset
min_size = min(len(true_csv), len(fake_csv))

# Undersample the larger dataset to match the size of the smaller dataset
df_true_undersampled = true_csv.sample(n=min_size, random_state=42)
df_fake_undersampled = fake_csv.sample(n=min_size, random_state=42)

# Concatenate the undersampled datasets
df = pd.concat([df_true_undersampled, df_fake_undersampled], ignore_index=True)

# Shuffle the concatenated dataset
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Now df_equal_size contains both datasets of equal size

In [ ]:
df.head()

,text,label
0,রাঙামাটির নানিয়ারচরে সন্ত্রাসীদের গুলিতে দুই ...,1
1,দুইটা বিড়ালের মধ্যে চ্যাট হচ্ছে-১ম বিড়াল: ম্যা...,0
2,অভিযুক্তের বিরুদ্ধে এফআইআর নেওয়ার জন্য আক্রান্...,0
3,আওয়ামী লীগ সাধারণ সমপাদক ওবায়দুল কাদের বলেছেন ...,1
4,টঙ্গীতে ন্যাশনাল ফ্যান কারখানায় হিট চেম্বার বি...,1


In [ ]:
df.tail()

,text,label
3476,Shareनई दिल्‍ली (Vishvas News)। सोशल मीडिया पर...,0
3477,Shareविश्वास न्यूज (नई दिल्ली)। सोशल मीडिया पर...,0
3478,Shareनई दिल्ली (विश्वास न्यूज)। एक यूट्यूब चैन...,0
3479,Shareनई दिल्‍ली (Vishvas News)। सोशल मीडिया पर...,0
3480,Shareनई दिल्‍ली (Vishvas News)। सोशल मीडिया मे...,0


In [ ]:
df.shape

(5334, 2)

In [6]:
duplicate_rows = df[df.duplicated()]
print(duplicate_rows)
df = pd.concat([true_csv, fake_csv], axis=0).drop_duplicates()


Empty DataFrame
Columns: [text, label]
Index: []


In [ ]:
df.isnull().sum()

text     0
label    0
dtype: int64

In [ ]:
df = df.sample(frac = 1)

In [ ]:
df.head()

,text,label
0,রাঙামাটির নানিয়ারচরে সন্ত্রাসীদের গুলিতে দুই ...,1
1,দুইটা বিড়ালের মধ্যে চ্যাট হচ্ছে-১ম বিড়াল: ম্যা...,0
2,অভিযুক্তের বিরুদ্ধে এফআইআর নেওয়ার জন্য আক্রান্...,0
3,আওয়ামী লীগ সাধারণ সমপাদক ওবায়দুল কাদের বলেছেন ...,1
4,টঙ্গীতে ন্যাশনাল ফ্যান কারখানায় হিট চেম্বার বি...,1


In [ ]:
(df['label'] == 1).sum()


2667

In [ ]:
df

,text,label
0,রাঙামাটির নানিয়ারচরে সন্ত্রাসীদের গুলিতে দুই ...,1
1,দুইটা বিড়ালের মধ্যে চ্যাট হচ্ছে-১ম বিড়াল: ম্যা...,0
2,অভিযুক্তের বিরুদ্ধে এফআইআর নেওয়ার জন্য আক্রান্...,0
3,আওয়ামী লীগ সাধারণ সমপাদক ওবায়দুল কাদের বলেছেন ...,1
4,টঙ্গীতে ন্যাশনাল ফ্যান কারখানায় হিট চেম্বার বি...,1
...,...,...
5329,সাবেক রাষ্ট্রপতি জাতীয় পার্টির চেয়ারম্যান হুসে...,0
5330,এইচএসসি পরীক্ষার ফলাফল ঘোষণার জন্য প্রথমে ২১ জ...,0
5331,বরিশালই হতে যাচ্ছে দ্বিতীয় সিঙ্গাপুর! উন্নয়ন...,0
5332,"কম্পিউটারের আবিষ্কারক কে, এই নিয়ে আছে নানান মত...",0


In [7]:
def wordopt(text):
    # text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [ ]:
# from collections.abc import Iterable
# import inltk
# from inltk.inltk import setup
# from inltk import setup
# setup('hi')
# print(dir(inltk))



In [ ]:
# !pip show inltk

In [ ]:
# import inltk
# print(dir(inltk))


In [8]:
# df_equal_size['text'] = df_equal_size['text'].apply(wordopt)
df['text'] = df['text'].apply(wordopt)


In [9]:
X = df['text']
y= df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# X = df_equal_size['text']
# y= df_equal_size['label']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
tfidf_vectorizer = TfidfVectorizer()
# tfidf_matrix = tfidf_vectorizer.fit_transform(df['text'])
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [ ]:
# print(tfidf_matrix)

In [ ]:
# tfidf_matrix.shape

In [ ]:
# tfidf_matrix[0, 0]

In [ ]:
# tfidf_matrix[0, 18996]

In [ ]:
# Convert sparse matrices to dense arrays
X_train_dense = X_train_tfidf.toarray()
X_test_dense = X_test_tfidf.toarray()


In [ ]:
input_dim=X_train_dense.shape[1]
print(input_dim)
# X_train_dense.shape()


7274


In [ ]:
# LSTM/BiLSTM model: accuracy=56%

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=X_train_dense.shape[1], output_dim=128, input_length=X_train_dense.shape[1]))
model.add(Bidirectional(LSTM(units=128)))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


model.fit(X_train_dense, y_train, epochs=10, batch_size=32, validation_data=(X_test_dense, y_test))

# Evaluate model
loss, accuracy = model.evaluate(X_test_dense, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

Epoch 1/10
134/134 [==============================] - 71s 474ms/step - loss: 0.6942 - accuracy: 0.5004 - val_loss: 0.6926 - val_accuracy: 0.5286
Epoch 2/10
 54/134 [===========>..................] - ETA: 33s - loss: 0.6934 - accuracy: 0.5081

KeyboardInterrupt: 

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 18326, 128)        2345728   
                                                                 
 bidirectional (Bidirection  (None, 256)               263168    
 al)                                                             
                                                                 
 dense (Dense)               (None, 1)                 257       
                                                                 
Total params: 2609153 (9.95 MB)
Trainable params: 2609153 (9.95 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
#CNN Model: accuracy=58%

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=X_train_dense.shape[1], output_dim=128, input_length=X_train_dense.shape[1]))
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train model
model.fit(X_train_dense, y_train, epochs=10, batch_size=32, validation_data=(X_test_dense, y_test))

# Evaluate model
loss, accuracy = model.evaluate(X_test_dense, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

Epoch 1/10
217/217 [==============================] - 51s 206ms/step - loss: 0.6765 - accuracy: 0.5989 - val_loss: 0.6783 - val_accuracy: 0.5883
Epoch 2/10
217/217 [==============================] - 45s 208ms/step - loss: 0.6740 - accuracy: 0.6003 - val_loss: 0.6775 - val_accuracy: 0.5883
Epoch 3/10
217/217 [==============================] - 43s 197ms/step - loss: 0.6738 - accuracy: 0.6003 - val_loss: 0.6775 - val_accuracy: 0.5883
Epoch 4/10
217/217 [==============================] - 45s 207ms/step - loss: 0.6733 - accuracy: 0.6003 - val_loss: 0.6783 - val_accuracy: 0.5883
Epoch 5/10
196/217 [==========================>...] - ETA: 3s - loss: 0.6736 - accuracy: 0.5993

KeyboardInterrupt: 

In [ ]:
## cnn + lstm: accuracy=59%

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=X_train_dense.shape[1], output_dim=128, input_length=X_train_dense.shape[1]))
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(units=128))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train model
model.fit(X_train_dense, y_train, epochs=10, batch_size=32, validation_data=(X_test_dense, y_test))

# Evaluate model
loss, accuracy = model.evaluate(X_test_dense, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

Epoch 1/10
 31/217 [===>..........................] - ETA: 1:11 - loss: 0.6784 - accuracy: 0.5988

KeyboardInterrupt: 

In [ ]:
!pip install fasttext


In [ ]:
# import fasttext

In [ ]:
# # Split the combined DataFrame into training and testing sets
# train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# # Separate the text and label data for training and testing
# X_train = train_df['text']
# y_train = train_df['label']
# X_test = test_df['text']
# y_test = test_df['label']

In [ ]:
# print("Data type of y_train:", type(y_train))
# y_train_list = y_train.tolist()


Data type of y_train: <class 'pandas.core.series.Series'>


In [ ]:
# # Prepare training data in FastText format
# with open('train.txt', 'w') as f:
#     for i, text in enumerate(X_train_dense):
#         if i < len(y_train):  # Check if the index is within the bounds of y_train
#             label = '__label__' + str(y_train_list[i])
#             f.write(label + ' ' + ' '.join(map(str, text)) + '\n')
#         else:
#             print(f"Warning: Missing label for sample at index {i}")

# # Train the FastText model
# model = fasttext.train_supervised('train.txt', epoch=10, lr=1.0)

# # Prepare test data in FastText format
# with open('test.txt', 'w') as f:
#     for i, text in enumerate(X_test_dense):
#         f.write(' '.join(map(str, text)) + '\n')

# # Evaluate the model
# results = model.test('test.txt')
# print(f'Test Accuracy: {results[1]:.4f}')

Test Accuracy: nan


In [ ]:
print(len(y_train))

4267


In [ ]:
print(len(X_train_dense))

4267


In [ ]:
# FASTTEXT MODEL: accuracy=87%
import fasttext



def wordopt(text):
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W", " ", text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

df['text'] = df['text'].apply(wordopt)


X = df['text']
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert labels to the FastText format
def format_label(label):
    return '__label__' + str(label)

# Create training data with labels in FastText format
y_train_list = y_train.tolist()
formatted_labels_train = [format_label(label) for label in y_train_list]

# Write the training data to a file
with open('train.txt', 'w') as f:
    for text, label in zip(X_train, formatted_labels_train):
        f.write(label + ' ' + text + '\n')

# Create test data with labels in FastText format
y_test_list = y_test.tolist()
formatted_labels_test = [format_label(label) for label in y_test_list]

# Write the test data to a file
with open('test.txt', 'w') as f:
    for text, label in zip(X_test, formatted_labels_test):
        f.write(label + ' ' + text + '\n')

# Train the FastText model
model = fasttext.train_supervised(input='train.txt')

# Evaluate the model using FastText's built-in method
test_result = model.test('test.txt')
print(f'Number of examples: {test_result[0]}')
print(f'Precision: {test_result[1]}')
print(f'Recall: {test_result[2]}')

Number of examples: 1067
Precision: 0.8716026241799437
Recall: 0.8716026241799437


In [ ]:
pip install transformers datasets tensorflow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 

In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import DataCollatorWithPadding
# from datasets import Dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

Map:   0%|          | 0/6926 [00:00<?, ? examples/s]

Map:   0%|          | 0/1732 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ValueError: Could not interpret optimizer identifier: <keras.src.optimizers.adam.Adam object at 0x7839e1383070>

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Map:   0%|          | 0/6926 [00:00<?, ? examples/s]

Map:   0%|          | 0/1732 [00:00<?, ? examples/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
433/433 [==============================] - 283s 482ms/step - loss: 6.4683 - accuracy: 0.5741 - val_loss: 6.2775 - val_accuracy: 0.5883
Epoch 2/10
433/433 [==============================] - 196s 454ms/step - loss: 6.0944 - accuracy: 0.6003 - val_loss: 6.2775 - val_accuracy: 0.5883
Epoch 3/10
433/433 [==============================] - 196s 452ms/step - loss: 6.0944 - accuracy: 0.6003 - val_loss: 6.2775 - val_accuracy: 0.5883
Epoch 4/10
433/433 [==============================] - 196s 453ms/step - loss: 6.0944 - accuracy: 0.6003 - val_loss: 6.2775 - val_accuracy: 0.5883
Epoch 5/10
433/433 [==============================] - 197s 454ms/step - loss: 6.0944 - accuracy: 0.6003 - val_loss: 6.2775 - val_accuracy: 0.5883
Epoch 6/10
433/433 [==============================] - 196s 452ms/step - loss: 6.0944 - accuracy: 0.6003 - val_loss: 6.2775 - val_accuracy: 0.5883
Epoch 7/10
433/433 [==============================] - 196s 453ms/step - loss: 6.0944 - accuracy: 0.6003 - val_loss: 6.2775 -

In [ ]:
#muril:82%

In [ ]:
import numpy as np
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModel

# Load the MURIL tokenizer and model
model_name = "google/muril-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
muril_model = TFAutoModel.from_pretrained(model_name)



# Convert target variables to NumPy arrays
y_train = np.array(y_train)
y_test = np.array(y_test)

def tokenize_and_encode(texts, tokenizer, max_length=128):
    input_ids = []
    attention_masks = []

    for text in texts:
        encoded = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_length,
            padding='max_length',  # Ensure padding to max_length
            truncation=True,       # Ensure truncation to max_length
            return_attention_mask=True,
            return_tensors='tf'
        )

        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])

    return tf.concat(input_ids, axis=0), tf.concat(attention_masks, axis=0)

# Tokenize and encode the training data
X_train_encodings, X_train_masks = tokenize_and_encode(X_train, tokenizer)
X_test_encodings, X_test_masks = tokenize_and_encode(X_test, tokenizer)

# Extract embeddings from MURIL
def extract_embeddings(bert_model, input_ids, attention_masks):
    outputs = bert_model(input_ids, attention_mask=attention_masks)
    embeddings = outputs.last_hidden_state
    return embeddings

# Define a function to process in smaller batches
def batch_process(func, bert_model, input_ids, attention_masks, batch_size=2):
    results = []
    for i in range(0, input_ids.shape[0], batch_size):
        batch_input_ids = input_ids[i:i + batch_size]
        batch_attention_masks = attention_masks[i:i + batch_size]
        result = func(bert_model, batch_input_ids, batch_attention_masks)
        results.append(result)
    return tf.concat(results, axis=0)

X_train_embeddings = batch_process(extract_embeddings, muril_model, X_train_encodings, X_train_masks)
X_test_embeddings = batch_process(extract_embeddings, muril_model, X_test_encodings, X_test_masks)

# Free up memory by deleting the input encodings and masks
del X_train_encodings, X_train_masks, X_test_encodings, X_test_masks

# Define your CNN/LSTM/BiLSTM model
model = tf.keras.Sequential()
model.add(tf.keras.layers.Input(shape=(X_train_embeddings.shape[1], X_train_embeddings.shape[2])))
model.add(tf.keras.layers.Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
model.add(tf.keras.layers.LSTM(units=128))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

# Compile model with mixed precision policy
policy = tf.keras.mixed_precision.Policy('mixed_float16')
tf.keras.mixed_precision.set_global_policy(policy)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train model with reduced batch size
model.fit(X_train_embeddings, y_train, epochs=10, batch_size=2, validation_data=(X_test_embeddings, y_test))

# Evaluate model
loss, accuracy = model.evaluate(X_test_embeddings, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')


Some layers from the model checkpoint at google/muril-base-cased were not used when initializing TFBertModel: ['mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertModel were not initialized from the model checkpoint at google/muril-base-cased and are newly initialized: ['bert/pooler/dense/kernel:0', 'bert/pooler/dense/bias:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
2134/2134 [==============================] - 197s 90ms/step - loss: 0.4985 - accuracy: 0.7406 - val_loss: 0.4804 - val_accuracy: 0.7891
Epoch 2/10
2134/2134 [==============================] - 182s 86ms/step - loss: 0.4674 - accuracy: 0.7734 - val_loss: 0.4505 - val_accuracy: 0.8097
Epoch 3/10
2134/2134 [==============================] - 183s 86ms/step - loss: 0.4272 - accuracy: 0.7989 - val_loss: 0.3971 - val_accuracy: 0.8238
Epoch 4/10
2134/2134 [==============================] - 183s 86ms/step - loss: 0.4049 - accuracy: 0.8156 - val_loss: 0.3871 - val_accuracy: 0.8341
Epoch 5/10
2134/2134 [==============================] - 185s 87ms/step - loss: 0.3880 - accuracy: 0.8296 - val_loss: 0.3819 - val_accuracy: 0.8332
Epoch 6/10
2134/2134 [==============================] - 182s 85ms/step - loss: 0.3849 - accuracy: 0.8277 - val_loss: 0.3773 - val_accuracy: 0.8360
Epoch 7/10
2134/2134 [==============================] - 182s 85ms/step - loss: 0.3697 - accuracy: 0.8355 - val_loss: 0

In [ ]:
#bert:91.3%(CNN),87%(LSTM)

In [12]:
from transformers import BertTokenizer

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# Tokenize the text data
def tokenize_text(texts, tokenizer, max_len=128):
    input_ids = []
    attention_masks = []

    for text in texts:
        encoded_dict = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_len,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='tf',
            truncation=True
        )

        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])

    return tf.concat(input_ids, axis=0), tf.concat(attention_masks, axis=0)

X_train_tokens, X_train_masks = tokenize_text(X_train, tokenizer)
X_test_tokens, X_test_masks = tokenize_text(X_test, tokenizer)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [13]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, GlobalMaxPooling1D, Dense, Dropout

def create_cnn_model(input_shape):
    input_ids = Input(shape=input_shape, dtype='int32')
    attention_masks = Input(shape=input_shape, dtype='int32')

    embedding_layer = tf.keras.layers.Embedding(input_dim=tokenizer.vocab_size, output_dim=128)(input_ids)

    conv_layer = Conv1D(filters=128, kernel_size=5, activation='relu')(embedding_layer)
    pooling_layer = GlobalMaxPooling1D()(conv_layer)

    dropout_layer = Dropout(0.5)(pooling_layer)
    output_layer = Dense(1, activation='sigmoid')(dropout_layer)

    model = Model(inputs=[input_ids, attention_masks], outputs=output_layer)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

cnn_model = create_cnn_model((128,))
cnn_model.summary()
cnn_model.fit([X_train_tokens, X_train_masks], y_train, batch_size=32, epochs=3, validation_split=0.2)
cnn_model.evaluate([X_test_tokens, X_test_masks], y_test)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 128)]                0         []                            
                                                                                                  
 embedding (Embedding)       (None, 128, 128)             1530201   ['input_5[0][0]']             
                                                          6                                       
                                                                                                  
 conv1d (Conv1D)             (None, 124, 128)             82048     ['embedding[0][0]']           
                                                                                                  
 global_max_pooling1d (Glob  (None, 128)                  0         ['conv1d[0][0]']          

[0.23260515928268433, 0.9130434989929199]

In [14]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Embedding

def create_lstm_model(input_shape):
    input_ids = Input(shape=input_shape, dtype='int32')
    attention_masks = Input(shape=input_shape, dtype='int32')

    embedding_layer = Embedding(input_dim=tokenizer.vocab_size, output_dim=128)(input_ids)

    lstm_layer = LSTM(128)(embedding_layer)

    dropout_layer = Dropout(0.5)(lstm_layer)
    output_layer = Dense(1, activation='sigmoid')(dropout_layer)

    model = Model(inputs=[input_ids, attention_masks], outputs=output_layer)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

lstm_model = create_lstm_model((128,))
lstm_model.summary()
lstm_model.fit([X_train_tokens, X_train_masks], y_train, batch_size=32, epochs=3, validation_split=0.2)
lstm_model.evaluate([X_test_tokens, X_test_masks], y_test)


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_7 (InputLayer)        [(None, 128)]                0         []                            
                                                                                                  
 embedding_1 (Embedding)     (None, 128, 128)             1530201   ['input_7[0][0]']             
                                                          6                                       
                                                                                                  
 lstm (LSTM)                 (None, 128)                  131584    ['embedding_1[0][0]']         
                                                                                                  
 dropout_1 (Dropout)         (None, 128)                  0         ['lstm[0][0]']          

[0.3555295467376709, 0.8731457591056824]